# Lab 1. Modelling and Optimization

In [2]:
# Import Libraries
import pulp

# Excercise 1. Glass Manufacturing Company

* You are planning the production at a glass manufacturer.
* This manufacturer only produces wine and beer glasses.
* There is a maximum production capacity of 60 hours.
* Each batch of wine and beer glasses takes 6 and 5 hours respectively.
* The warehouse has a maximum capacity of 150 m3 spaces.
* Each batch of the wine and beer glasses takes 10 and 20 m3 respectively.
* The production equipment can only make full batches, no partial batches.
* We only have orders for 6 batches of wine glasses.
* Each batch of wine glasses earns a profit of 5 USD and the beer 4.5 USD.
* The objective is to maximize the profit for the manufacturer.

In [3]:
# 1. Initialize Model
GMC_problem = pulp.LpProblem("Glass_Manufacturing_Profit_Maximization", pulp.LpMaximize)

# 2. Define Decision Variables
wine = pulp.LpVariable("Wine_Glasses_Batches", lowBound=0, cat=pulp.LpInteger)
beer = pulp.LpVariable("Beer_Glasses_Batches", lowBound=0, cat=pulp.LpInteger)
# 3. Define Objective Function
profit = 5 * wine + 4.5 * beer
GMC_problem += profit
# 4. Define Constraints
GMC_problem += 6 * wine + 5 * beer <= 60 
GMC_problem += 10 * wine + 20 * beer <= 150
GMC_problem += wine <= 6

# 5. Solve Model
GMC_problem.solve()

# 6. Print Results
optimal_wine = wine.varValue
optimal_beer = beer.varValue
maximum_profit = pulp.value(GMC_problem.objective)
print(f"Number of wine glass batches to produce: {optimal_wine}")
print(f"Number of beer glass batches to produce: {optimal_beer}")
print(f"Maximum profit: ${maximum_profit:.2f}")

Number of wine glass batches to produce: 6.0
Number of beer glass batches to produce: 4.0
Maximum profit: $48.00


### Exercise 2. Logistic Planning Problem

* You are consulting for kitchen oven manufacturer helping to plan their logistics for next month.
* There are two warehouse locations (Lyon and Bilbao)
* There are four regional customer locations (Portugal, Spain, France, Italy)
* The expected demand next month are:
* for Italy it is 1,800
* for France it is 1,200
* for the Spain it is 1,100
* and for Portugal it is 1000.
* The cost for shipping each of the warehouse locations to the regional customer's are the next:
* Italy to Bilbao 211 USD, France to Bilbao 232 USD, Spain to Bilbao 240 USD, Portugal to Bilbao 300 USD
* Italy to Lyon 232 USD, France to Lyon 212 USD, Spain to Lyon 230 USD, Portugal to Lyon 280 USD.
* Your goal is to fullfill the regional demand at the lowest price.

In [6]:
# 1. Initialize Model
LPP_problem = pulp.LpProblem("Logistics_Problem", pulp.LpMinimize)
# 2. Define Decision Variables
warehouses = ["Lyon", "Bilbao"]
customers_region = ["Portugal", "Spain", "France", "Italy"]
shipping_costs = {
    ("Italy", "Bilbao"): 211,
    ("France", "Bilbao"): 232,
    ("Spain", "Bilbao"): 240,
    ("Portugal", "Bilbao"): 300,
    ("Italy", "Lyon"): 232,
    ("France", "Lyon"): 212,
    ("Spain", "Lyon"): 230,
    ("Portugal", "Lyon"): 280,
}
x = pulp.LpVariable.dicts("ship", ((i, j) for i in customers_region for j in warehouses), lowBound=0, cat=pulp.LpInteger)
# 3. Define Objective Function
LPP_problem += pulp.lpSum(x[i, j] * shipping_costs[i, j]  for i in customers_region for j in warehouses)

# 4. Define Constraints
demand = {
    "Italy": 1800,
    "France": 1200,
    "Spain": 1100,
    "Portugal": 1000,
}
for i in customers_region:
    LPP_problem += pulp.lpSum(x[i, j] for j in warehouses) == demand[i]

# 5. Solve Model
LPP_problem.solve()

# 6. Print Results
print("Optimal Solution:")
for i in warehouses:
    for j in customers_region:
        print(f"Ship {x[j, i].varValue} ovens from {i} to {j}")

print(f"Total Cost: ${pulp.value(LPP_problem.objective)}")

Optimal Solution:
Ship 1000.0 ovens from Lyon to Portugal
Ship 1100.0 ovens from Lyon to Spain
Ship 1200.0 ovens from Lyon to France
Ship 0.0 ovens from Lyon to Italy
Ship 0.0 ovens from Bilbao to Portugal
Ship 0.0 ovens from Bilbao to Spain
Ship 0.0 ovens from Bilbao to France
Ship 1800.0 ovens from Bilbao to Italy
Total Cost: $1167200.0
